<h1 align='center' style='color:black'> Customer Segmentation </h1>

<img src='Customer Segmentation.PNG' />

## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from datetime import date
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
#from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,confusion_matrix,precision_score,recall_score,accuracy_score,f1_score
%matplotlib inline
import scipy.stats 
from scipy.stats import chi2_contingency

## Loading Data

In [2]:
# Showing all the columns
pd.set_option('display.max_columns',None) 
#pd.set_option('display.max_rows',None)

In [3]:
data=pd.read_csv('cust_seg.csv')
# Printing the data
data

,Unnamed: 0,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,NaN,1.0,A,S,N,NaN,KHL,N,1.0,29.0,MALAGA,1.0,87218.10,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
1,1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,NaN,1.0,I,S,S,NaN,KHE,N,1.0,13.0,CIUDAD REAL,0.0,35548.74,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
2,2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,NaN,1.0,I,S,N,NaN,KHE,N,1.0,13.0,CIUDAD REAL,0.0,122179.11,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,NaN,1.0,I,S,N,NaN,KHD,N,1.0,50.0,ZARAGOZA,0.0,119775.54,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
4,4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,NaN,1.0,A,S,N,NaN,KHE,N,1.0,50.0,ZARAGOZA,1.0,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,999995,2015-02-28,1183296,N,ES,H,27,2013-09-25,0.0,22,1.0,NaN,1.0,A,S,N,NaN,KFC,N,1.0,28.0,MADRID,1.0,55516.98,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1
999996,999996,2015-02-28,1183295,N,ES,H,56,2013-09-25,0.0,22,1.0,NaN,1.0,A,S,N,NaN,KFC,N,1.0,15.0,"CORUÑA, A",1.0,75654.84,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
999997,999997,2015-02-28,1183294,N,ES,V,39,2013-09-25,0.0,22,1.0,NaN,1.0,A,S,N,NaN,KFC,N,1.0,15.0,"CORUÑA, A",1.0,80634.87,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1
999998,999998,2015-02-28,1183293,N,ES,V,36,2013-09-25,0.0,22,1.0,NaN,1.0,A,S,S,NaN,KFC,N,1.0,28.0,MADRID,1.0,57818.46,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1


In [4]:
# Putting the data into dataframe
df= pd.DataFrame(data)
df.head()

,Unnamed: 0,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,NaN,1.0,A,S,N,NaN,KHL,N,1.0,29.0,MALAGA,1.0,87218.10,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
1,1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,NaN,1.0,I,S,S,NaN,KHE,N,1.0,13.0,CIUDAD REAL,0.0,35548.74,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
2,2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,NaN,1.0,I,S,N,NaN,KHE,N,1.0,13.0,CIUDAD REAL,0.0,122179.11,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,NaN,1.0,I,S,N,NaN,KHD,N,1.0,50.0,ZARAGOZA,0.0,119775.54,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
4,4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,NaN,1.0,A,S,N,NaN,KHE,N,1.0,50.0,ZARAGOZA,1.0,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0


## EDA (Exploratory Data Analysis)

## Printing the Shape

In [5]:
print ("The Shape is : ",data.shape)

The Shape is :  (1000000, 48)


## Printing the Columns Names

In [6]:
print(data.columns)

Index(['Unnamed: 0', 'fecha_dato', 'ncodpers', 'ind_empleado',
       'pais_residencia', 'sexo', 'age', 'fecha_alta', 'ind_nuevo',
       'antiguedad', 'indrel', 'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes',
       'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'tipodom',
       'cod_prov', 'nomprov', 'ind_actividad_cliente', 'renta',
       'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1'],
      dtype='object')


## Removing 'Unnamed: 0' and  'fecha_dato' column

In [7]:
data.drop(['Unnamed: 0'],axis=1,inplace=True)
# Printing the data after removing 'Unnamed: 0' column
data.head(2)

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,NaN,1.0,A,S,N,NaN,KHL,N,1.0,29.0,MALAGA,1.0,87218.10,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,NaN,1.0,I,S,S,NaN,KHE,N,1.0,13.0,CIUDAD REAL,0.0,35548.74,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0


In [8]:
#Drop the fecha_dato column because it was used for partition of the data 
data.drop(['fecha_dato'],axis=1,inplace=True)
# Printing the data after removing 'fecha_dato' column
data.head(2)

,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,NaN,1.0,A,S,N,NaN,KHL,N,1.0,29.0,MALAGA,1.0,87218.10,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
1,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,NaN,1.0,I,S,S,NaN,KHE,N,1.0,13.0,CIUDAD REAL,0.0,35548.74,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0


## Data Describtion

In [9]:
# Describibg the customer's age 
data['age'].describe()

count     1000000
unique        115
top            22
freq        51017
Name: age, dtype: object

In [ ]:
# Describibg the customer's sex 
data['sexo'].describe()

In [ ]:
# Describibg the customer relation type at the beginning of the month 
data['tiprel_1mes'].describe()

In [ ]:
# Describibg the residence index (S (Yes) or N (No) if the residence country is the same than the bank country)
data['indresi'].describe()

In [ ]:
# Describibg the foreigner index (S (Yes) or N (No) if the customer's birth country is different than the bank country)
data['indext'].describe()

In [ ]:
# Describibg the Spouse index (1 if the customer is spouse of an employee)
data['conyuemp'].describe()

In [ ]:
# Describibg the deceased index
data['indfall'].describe()

In [ ]:
data['pais_residencia'].describe()

## Looking for Missing Values

In [10]:
data.isnull().sum()

ncodpers                      0
ind_empleado              10782
pais_residencia           10782
sexo                      10786
age                           0
fecha_alta                10782
ind_nuevo                 10782
antiguedad                    0
indrel                    10782
ult_fec_cli_1t           998899
indrel_1mes               10782
tiprel_1mes               10782
indresi                   10782
indext                    10782
conyuemp                 999822
canal_entrada             10861
indfall                   10782
tipodom                   10782
cod_prov                  17734
nomprov                   17734
ind_actividad_cliente     10782
renta                    175183
ind_ahor_fin_ult1             0
ind_aval_fin_ult1             0
ind_cco_fin_ult1              0
ind_cder_fin_ult1             0
ind_cno_fin_ult1              0
ind_ctju_fin_ult1             0
ind_ctma_fin_ult1             0
ind_ctop_fin_ult1             0
ind_ctpp_fin_ult1             0
ind_deco

## Printing Data Type and Capacity of The Data on The Memory

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 46 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   ncodpers               1000000 non-null  int64  
 1   ind_empleado           989218 non-null   object 
 2   pais_residencia        989218 non-null   object 
 3   sexo                   989214 non-null   object 
 4   age                    1000000 non-null  object 
 5   fecha_alta             989218 non-null   object 
 6   ind_nuevo              989218 non-null   float64
 7   antiguedad             1000000 non-null  object 
 8   indrel                 989218 non-null   float64
 9   ult_fec_cli_1t         1101 non-null     object 
 10  indrel_1mes            989218 non-null   float64
 11  tiprel_1mes            989218 non-null   object 
 12  indresi                989218 non-null   object 
 13  indext                 989218 non-null   object 
 14  conyuemp           

## Some Statistics

In [ ]:
# Printing the values of (Sex) column with its domain
data['sexo'].value_counts()

In [ ]:
# Printing the values of (deceased index) column with its domain
data['indfall'].value_counts()

In [ ]:
# Printing the values of (Spouse index) column with its domain
data['conyuemp'].value_counts()

In [ ]:
# Printing the values of (foreigner index) column with its domain
data['indext'].value_counts()

In [ ]:
# Printing the values of (residence index) column with its domain
data['indresi'].value_counts()

In [ ]:
# Printing the values of (Customer relation type at the beginning of the month) column with its domain
data['tiprel_1mes'].value_counts()

## Finding the Duplicate Rows

In [13]:
#store all duplicates
df_duplicates = data[data.duplicated(['ncodpers','age','sexo','nomprov','renta','fecha_alta'])]

In [14]:
#drop all duplicates
newdata = data.drop_duplicates(subset=['ncodpers','age','sexo','nomprov','renta','fecha_alta'],keep = 'last').reset_index(drop = True)

In [15]:
newdata

,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,NaN,1.0,A,S,N,NaN,KHL,N,1.0,29.0,MALAGA,1.0,87218.10,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
1,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,NaN,1.0,I,S,S,NaN,KHE,N,1.0,13.0,CIUDAD REAL,0.0,35548.74,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
2,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,NaN,1.0,I,S,N,NaN,KHE,N,1.0,13.0,CIUDAD REAL,0.0,122179.11,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,NaN,1.0,I,S,N,NaN,KHD,N,1.0,50.0,ZARAGOZA,0.0,119775.54,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
4,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,NaN,1.0,A,S,N,NaN,KHE,N,1.0,50.0,ZARAGOZA,1.0,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626154,1183296,N,ES,H,27,2013-09-25,0.0,22,1.0,NaN,1.0,A,S,N,NaN,KFC,N,1.0,28.0,MADRID,1.0,55516.98,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1
626155,1183295,N,ES,H,56,2013-09-25,0.0,22,1.0,NaN,1.0,A,S,N,NaN,KFC,N,1.0,15.0,"CORUÑA, A",1.0,75654.84,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
626156,1183294,N,ES,V,39,2013-09-25,0.0,22,1.0,NaN,1.0,A,S,N,NaN,KFC,N,1.0,15.0,"CORUÑA, A",1.0,80634.87,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1
626157,1183293,N,ES,V,36,2013-09-25,0.0,22,1.0,NaN,1.0,A,S,S,NaN,KFC,N,1.0,28.0,MADRID,1.0,57818.46,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1


In [16]:
#check
newdata.duplicated(['ncodpers','age','sexo','nomprov','renta','fecha_alta']).sum()

0

## Handling the  Missing Values

In [17]:
#drop all variables that have more than 80% of the data missing (conyuemp,ult_fec_cli_1t)
newdata.drop(['conyuemp'],axis=1,inplace=True)
newdata.drop(['ult_fec_cli_1t'],axis=1,inplace=True)
newdata.head(1)

,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,indrel_1mes,tiprel_1mes,indresi,indext,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,1.0,A,S,N,KHL,N,1.0,29.0,MALAGA,1.0,87218.1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0


In [18]:
#remove all rows with no age
newdata['age'].replace({" NA":""},inplace=True)
newdata['age'] = pd.to_numeric(newdata['age'])
newdata.dropna(subset = ['age'],inplace = True)
newdata.isnull().sum()

ncodpers                      0
ind_empleado                  0
pais_residencia               0
sexo                          2
age                           0
fecha_alta                    0
ind_nuevo                     0
antiguedad                    0
indrel                        0
indrel_1mes                   0
tiprel_1mes                   0
indresi                       0
indext                        0
canal_entrada                51
indfall                       0
tipodom                       0
cod_prov                   3682
nomprov                    3682
ind_actividad_cliente         0
renta                    104731
ind_ahor_fin_ult1             0
ind_aval_fin_ult1             0
ind_cco_fin_ult1              0
ind_cder_fin_ult1             0
ind_cno_fin_ult1              0
ind_ctju_fin_ult1             0
ind_ctma_fin_ult1             0
ind_ctop_fin_ult1             0
ind_ctpp_fin_ult1             0
ind_deco_fin_ult1             0
ind_deme_fin_ult1             0
ind_dela

In [19]:
#replace sex, canal_entrada, ind_nomina_ult1 and ind_nom_pens_ult1 using mode
newdata['ind_nom_pens_ult1'] = newdata['ind_nom_pens_ult1'].fillna(newdata['ind_nom_pens_ult1'].mode()[0])
newdata['ind_nomina_ult1'] =newdata['ind_nomina_ult1'].fillna(newdata['ind_nomina_ult1'].mode()[0])
newdata['sexo'] = newdata['sexo'].fillna(newdata['sexo'].mode()[0])

In [20]:
#replce renta (gross income of household) using the mean
newdata['renta'] = newdata['renta'].fillna(newdata['renta'].mean())

In [21]:
#replace the customer's country of resident by code 0 for countries outside europe
newdata['cod_prov'] = newdata['cod_prov'].fillna(0)
newdata['nomprov'] = newdata['nomprov'].fillna('NotEuropean')

In [22]:
#replace the customers channel used to join using the mode
newdata['canal_entrada'] = newdata['canal_entrada'].fillna(newdata['canal_entrada'].mode()[0])

In [23]:
newdata.isnull().sum()

ncodpers                 0
ind_empleado             0
pais_residencia          0
sexo                     0
age                      0
fecha_alta               0
ind_nuevo                0
antiguedad               0
indrel                   0
indrel_1mes              0
tiprel_1mes              0
indresi                  0
indext                   0
canal_entrada            0
indfall                  0
tipodom                  0
cod_prov                 0
nomprov                  0
ind_actividad_cliente    0
renta                    0
ind_ahor_fin_ult1        0
ind_aval_fin_ult1        0
ind_cco_fin_ult1         0
ind_cder_fin_ult1        0
ind_cno_fin_ult1         0
ind_ctju_fin_ult1        0
ind_ctma_fin_ult1        0
ind_ctop_fin_ult1        0
ind_ctpp_fin_ult1        0
ind_deco_fin_ult1        0
ind_deme_fin_ult1        0
ind_dela_fin_ult1        0
ind_ecue_fin_ult1        0
ind_fond_fin_ult1        0
ind_hip_fin_ult1         0
ind_plan_fin_ult1        0
ind_pres_fin_ult1        0
i

## Feature Fixing

In [24]:
newdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 619174 entries, 0 to 626158
Data columns (total 44 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ncodpers               619174 non-null  int64  
 1   ind_empleado           619174 non-null  object 
 2   pais_residencia        619174 non-null  object 
 3   sexo                   619174 non-null  object 
 4   age                    619174 non-null  float64
 5   fecha_alta             619174 non-null  object 
 6   ind_nuevo              619174 non-null  float64
 7   antiguedad             619174 non-null  object 
 8   indrel                 619174 non-null  float64
 9   indrel_1mes            619174 non-null  float64
 10  tiprel_1mes            619174 non-null  object 
 11  indresi                619174 non-null  object 
 12  indext                 619174 non-null  object 
 13  canal_entrada          619174 non-null  object 
 14  indfall                619174 non-nu

In [26]:
#Organising the variables according to what type they are.
date_vars = ['fecha_alta']

bionomial_vars =['sexo','ind_nuevo','indrel','indresi', 'indext','indfall','ind_actividad_cliente']

multicategorical_vars = ['ind_empleado','pais_residencia', 'cod_prov','tipodom', 'nomprov','canal_entrada', 'indrel_1mes','tiprel_1mes']

intcategorical_vars = ['ind_ahor_fin_ult1','ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1','ind_cno_fin_ult1',
                       'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1','ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1',
                       'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1','ind_fond_fin_ult1', 'ind_hip_fin_ult1', 
                       'ind_plan_fin_ult1','ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1','ind_valo_fin_ult1',
                       'ind_viv_fin_ult1','ind_nomina_ult1','ind_nom_pens_ult1', 'ind_recibo_ult1']
all_cat_vars = ['sexo','ind_nuevo','indrel','indresi', 'indext','indfall','ind_actividad_cliente',
               'ind_empleado','pais_residencia', 'cod_prov', 'nomprov','canal_entrada', 'indrel_1mes','tiprel_1mes',
               'ind_ahor_fin_ult1','ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1','ind_cno_fin_ult1',
                       'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1','ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1',
                       'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1','ind_fond_fin_ult1', 'ind_hip_fin_ult1', 
                       'ind_plan_fin_ult1','ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1','ind_valo_fin_ult1',
                       'ind_viv_fin_ult1','ind_nomina_ult1','ind_nom_pens_ult1', 'ind_recibo_ult1']

numeric_vars = ['ncodpers' ,'age','antiguedad', 'renta','fecha_alta_year']

float_vars = ['ind_nuevo','indrel','age','indrel_1mes','tipodom','cod_prov','ind_actividad_cliente', 'ind_nomina_ult1','ind_nom_pens_ult1']

In [27]:
newdata[float_vars] = newdata[float_vars].round(0).astype('int')
newdata['antiguedad'] = newdata['antiguedad'].astype('int')

In [28]:
newdata[bionomial_vars] = newdata[bionomial_vars].astype('category')
newdata[multicategorical_vars] = newdata[multicategorical_vars].astype('category')
newdata[intcategorical_vars] = newdata[intcategorical_vars].astype('category')

In [29]:
newdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 619174 entries, 0 to 626158
Data columns (total 44 columns):
 #   Column                 Non-Null Count   Dtype   
---  ------                 --------------   -----   
 0   ncodpers               619174 non-null  int64   
 1   ind_empleado           619174 non-null  category
 2   pais_residencia        619174 non-null  category
 3   sexo                   619174 non-null  category
 4   age                    619174 non-null  int32   
 5   fecha_alta             619174 non-null  object  
 6   ind_nuevo              619174 non-null  category
 7   antiguedad             619174 non-null  int32   
 8   indrel                 619174 non-null  category
 9   indrel_1mes            619174 non-null  category
 10  tiprel_1mes            619174 non-null  category
 11  indresi                619174 non-null  category
 12  indext                 619174 non-null  category
 13  canal_entrada          619174 non-null  category
 14  indfall             

In [ ]:
#checking how the data looks like
newdata.to_csv(r'outputdata.csv', index = False)

In [30]:
#Fix the date variables: fecha_alta [The date in which the customer become the first holder of contract at the bank]
#subtract each date with the max date and get the number of year since the first time 
newdata['fecha_alta'] = newdata['fecha_alta'].apply(lambda _:datetime.datetime.strptime(_,'%Y-%m-%d'))

In [31]:
max_date = datetime.datetime.strptime('2015-01-28','%Y-%m-%d')
newdata['fecha_alta_year'] = newdata['fecha_alta'].apply(lambda _:((max_date-_)/np.timedelta64(1,'Y')))

In [32]:
#convert to number of years 
newdata['fecha_alta_year'] = newdata['fecha_alta_year'].round(0).astype(int)

In [33]:
newdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 619174 entries, 0 to 626158
Data columns (total 45 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   ncodpers               619174 non-null  int64         
 1   ind_empleado           619174 non-null  category      
 2   pais_residencia        619174 non-null  category      
 3   sexo                   619174 non-null  category      
 4   age                    619174 non-null  int32         
 5   fecha_alta             619174 non-null  datetime64[ns]
 6   ind_nuevo              619174 non-null  category      
 7   antiguedad             619174 non-null  int32         
 8   indrel                 619174 non-null  category      
 9   indrel_1mes            619174 non-null  category      
 10  tiprel_1mes            619174 non-null  category      
 11  indresi                619174 non-null  category      
 12  indext                 619174 non-null  cate

In [34]:
#create a column for that counts the number of accounts each user has
newdata["number_of_accounts"] = newdata[intcategorical_vars].sum(axis=1).astype('int')

## Checking Normality of Variables

## Outliers Detection and Removing

In [35]:
#remove outliers in variable antiguedad
q1_antiguedad = newdata['antiguedad'].quantile(0.25)
q3_antiguedad = newdata['antiguedad'].quantile(0.75)
iqr_antiguedad = q3_antiguedad - q1_antiguedad
lowbound_antiguedad = q1_antiguedad- 1.5*iqr_antiguedad
upperbound_antiguedad = q3_antiguedad + 1.5*iqr_antiguedad

In [36]:
newdata = newdata[((newdata['antiguedad'] < lowbound_antiguedad)|(newdata['antiguedad'] > upperbound_antiguedad))==False]

In [37]:
#remove outliers in age 
q1_age = newdata['age'].quantile(0.25)
q3_age = newdata['age'].quantile(0.75)
iqr_age = q3_age - q1_age
lowbound_age = q1_age - 1.5*iqr_age
upperbound_age = q3_age + 1.5*iqr_age

In [38]:
newdata = newdata[((newdata['age'] < lowbound_age)|(newdata['age'] > upperbound_age))==False]

In [39]:
#remove outliers in renta
q1_renta = newdata['renta'].quantile(0.25)
q3_renta = newdata['renta'].quantile(0.75)
iqr_renta = q3_renta - q1_renta
lowbound_renta = q1_renta - 1.5*iqr_renta
upperbound_renta = q3_renta + 1.5*iqr_renta

In [40]:
newdata = newdata[((newdata['renta'] < lowbound_renta)|(newdata['renta'] > upperbound_renta))==False]

In [41]:
newdata

,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,indrel_1mes,tiprel_1mes,indresi,indext,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,fecha_alta_year,number_of_accounts
0,1375586,N,ES,H,35,2015-01-12,0,6,1,1,A,S,N,KHL,N,1,29,MALAGA,1,87218.100000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1050611,N,ES,V,23,2012-08-10,0,35,1,1,I,S,S,KHE,N,1,13,CIUDAD REAL,0,35548.740000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1
2,1050612,N,ES,V,23,2012-08-10,0,35,1,1,I,S,N,KHE,N,1,13,CIUDAD REAL,0,122179.110000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1
3,1050613,N,ES,H,22,2012-08-10,0,35,1,1,I,S,N,KHD,N,1,50,ZARAGOZA,0,119775.540000,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1
4,1050614,N,ES,V,23,2012-08-10,0,35,1,1,A,S,N,KHE,N,1,50,ZARAGOZA,1,134602.202738,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626154,1183296,N,ES,H,27,2013-09-25,0,22,1,1,A,S,N,KFC,N,1,28,MADRID,1,55516.980000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1
626155,1183295,N,ES,H,56,2013-09-25,0,22,1,1,A,S,N,KFC,N,1,15,"CORUÑA, A",1,75654.840000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
626156,1183294,N,ES,V,39,2013-09-25,0,22,1,1,A,S,N,KFC,N,1,15,"CORUÑA, A",1,80634.870000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2
626157,1183293,N,ES,V,36,2013-09-25,0,22,1,1,A,S,S,KFC,N,1,28,MADRID,1,57818.460000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2


## Checking Correlation - to see how much those variables  are related 

In [42]:
#relationship between the numeric variables 
index_numeric = ['age','antiguedad', 'renta','number_of_accounts']
df_numeric = newdata[index_numeric]
df_numeric.corr()  #make sense antiguedad describes customer seniority thus it is related to age.

,age,antiguedad,renta,number_of_accounts
age,1.000000,0.609699,0.148001,0.253102
antiguedad,0.609699,1.000000,0.185266,0.366929
renta,0.148001,0.185266,1.000000,0.115368
number_of_accounts,0.253102,0.366929,0.115368,1.000000


In [ ]:
#relationship between the categorical variables
#make use of the chi_squared test
#chi2_contingency(pd.crosstab(cat_data['sexo'],cat_data['ind_nuevo']))

In [44]:
def correlation(a,b):
    c,p,dof,expected = chi2_contingency(pd.crosstab(a,b))
    return(p)

In [ ]:
corr_df = pd.DataFrame()
plist = []
for name in all_cat_vars:
    for i in all_cat_vars:
        p = correlation(newdata[name], newdata[i])
        plist.append(p)
    corr_df[name]=plist
    plist = []

corr_df    

In [45]:
correlation(newdata['nomprov'],newdata['cod_prov'])

0.0

## Label Encoding

In [46]:
#label Encoding: Normial variables 
#one hot encoding for sexo, indresi, indext, indfall, tipodom,ind_nuevo 
hotcode_label = ['sexo', 'indresi', 'indext', 'indfall', 'tipodom', 'ind_nuevo']
newdata = pd.get_dummies(data = newdata, columns = hotcode_label)
newdata.head() #tipodom has one category 

,ncodpers,ind_empleado,pais_residencia,age,fecha_alta,antiguedad,indrel,indrel_1mes,tiprel_1mes,canal_entrada,cod_prov,nomprov,ind_actividad_cliente,renta,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,fecha_alta_year,number_of_accounts,sexo_H,sexo_V,indresi_N,indresi_S,indext_N,indext_S,indfall_N,indfall_S,tipodom_1,ind_nuevo_0,ind_nuevo_1
0,1375586,N,ES,35,2015-01-12,6,1,1,A,KHL,29,MALAGA,1,87218.100000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,1,0,1,1,0
1,1050611,N,ES,23,2012-08-10,35,1,1,I,KHE,13,CIUDAD REAL,0,35548.740000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,1,0,1,0,1,1,0,1,1,0
2,1050612,N,ES,23,2012-08-10,35,1,1,I,KHE,13,CIUDAD REAL,0,122179.110000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,1,0,1,1,0,1,0,1,1,0
3,1050613,N,ES,22,2012-08-10,35,1,1,I,KHD,50,ZARAGOZA,0,119775.540000,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,1,0,0,1,1,0,1,0,1,1,0
4,1050614,N,ES,23,2012-08-10,35,1,1,A,KHE,50,ZARAGOZA,1,134602.202738,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,1,0,1,1,0,1,0,1,1,0


In [47]:
#Make use of frequency encoding for cod_prov, nomprov, pais_residencia
freq_cod_prov = (newdata.groupby('cod_prov').size())/len(newdata)
freq_nomprov = (newdata.groupby('nomprov').size())/len(newdata)
freq_pais_residencia = (newdata.groupby('pais_residencia').size())/len(newdata)
freq_canal_entrada = (newdata.groupby('canal_entrada').size())/len(newdata)

newdata.loc[:,'cod_prov'] = newdata['cod_prov'].map(freq_cod_prov)
newdata.loc[:,'nomprov'] = newdata['nomprov'].map(freq_nomprov)
newdata.loc[:,'pais_residencia'] = newdata['pais_residencia'].map(freq_pais_residencia)
newdata.loc[:,'canal_entrada'] = newdata['canal_entrada'].map(freq_canal_entrada)

In [48]:
#label Encoding: Ordinal variables 
#indrel, indrel_1mes, tiprel_1mes
tiprel_1mes_dict = {'A':3,'I':1,'P':2}
indrel_dict = {1:2,99:1}
indrel_1mes_dict = {1:3,2:2,3:1}


newdata['tiprel_1mes']=newdata.tiprel_1mes.map(tiprel_1mes_dict)
newdata['indrel']=newdata.indrel.map(indrel_dict)
newdata['indrel_1mes']=newdata.indrel_1mes.map(indrel_1mes_dict)


## Feature Selection

In [49]:
#Removing: tipodom_1 (has one class only), fecha_alta(date has been manipulated), nom_prov (is the same as cod_prov), 
#int_categorical variables

drop_cols = ['tipodom_1','fecha_alta','nomprov','ind_ahor_fin_ult1','ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1',
             'ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1','ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
             'ind_deco_fin_ult1','ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1','ind_fond_fin_ult1', 
             'ind_hip_fin_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1',
             'ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1','ind_nom_pens_ult1', 'ind_recibo_ult1']
newdata.drop(drop_cols,axis=1,inplace=True)


AttributeError: 'NoneType' object has no attribute 'head'

In [54]:
newdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 567592 entries, 0 to 626158
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype   
---  ------                 --------------   -----   
 0   ncodpers               567592 non-null  int64   
 1   ind_empleado           567592 non-null  category
 2   pais_residencia        567592 non-null  float64 
 3   age                    567592 non-null  int32   
 4   antiguedad             567592 non-null  int32   
 5   indrel                 567592 non-null  category
 6   indrel_1mes            567592 non-null  category
 7   tiprel_1mes            567592 non-null  category
 8   canal_entrada          567592 non-null  float64 
 9   cod_prov               567592 non-null  category
 10  ind_actividad_cliente  567592 non-null  category
 11  renta                  567592 non-null  float64 
 12  fecha_alta_year        567592 non-null  int32   
 13  number_of_accounts     567592 non-null  int32   
 14  sexo_H              

## Data Visualization

## Feature Scaling

In [55]:
newdata

,ncodpers,ind_empleado,pais_residencia,age,antiguedad,indrel,indrel_1mes,tiprel_1mes,canal_entrada,cod_prov,ind_actividad_cliente,renta,fecha_alta_year,number_of_accounts,sexo_H,sexo_V,indresi_N,indresi_S,indext_N,indext_S,indfall_N,indfall_S,ind_nuevo_0,ind_nuevo_1
0,1375586,N,0.993531,35,6,2,3,3,0.001212,0.029271,1,87218.100000,0,1,1,0,0,1,1,0,1,0,1,0
1,1050611,N,0.993531,23,35,2,3,1,0.357863,0.009669,0,35548.740000,2,1,0,1,0,1,0,1,1,0,1,0
2,1050612,N,0.993531,23,35,2,3,1,0.357863,0.009669,0,122179.110000,2,1,0,1,0,1,1,0,1,0,1,0
3,1050613,N,0.993531,22,35,2,3,1,0.009877,0.026928,0,119775.540000,2,1,1,0,0,1,1,0,1,0,1,0
4,1050614,N,0.993531,23,35,2,3,3,0.357863,0.026928,1,134602.202738,2,1,0,1,0,1,1,0,1,0,1,0


## Model Building

In [ ]:
#hierarchical clustering, fuzzy clustering, gaussian mixture model, kmeans

## Model Evaluation

## Model Saving